# Plant Species Classification

The Aarhus University Signal Processing group, in collaboration with University of Southern Denmark, has released a dataset containing images of unique plants belonging to 12 species at several growth stages.

The ability to differentiate between a weed and a crop seedling can mean better crop yields and better stewardship of the environment.

# Problem:
- Classify the image based on the pixel value.
- The result of the model should be the number which is represented by the pixel values.
- Use Supervised Learning method for it.

# Data:
- **label:** Total 12 unique values.
- **pixel0, pixel1, pixel2, ..., pixel782, pixel783:** Each value in these columns is between 0 and 255. Which represents the pixel intensity. 

In [1]:
%tensorflow_version 2.x
import tensorflow
tensorflow.__version__

'2.4.1'

In [4]:
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras import datasets, models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from google.colab.patches import cv2_imshow

from sklearn import svm
from sklearn.model_selection import train_test_split

## Loading the data


- images.npy and Label.csv 
- print shape of data
- visualize the images in dataset

# Data Pre-processing

- Normalization
- Gaussian Blurring
- Visualize data after pre-processing

# Make data compatible

- Convert labels to one-hot-vectors.
- Print the label for y_train[0].
- Split the dataset into training, testing, and validation set.
(First split images and labels into training and testing set with test_size = 0.3. Then further split test data
into test and validation set with test_size = 0.5)
-Check the shape of data, Reshape data into shapes compatible with Keras models if it’s not already. If it’s
already in the compatible shape, then comment in the notebook that it’s already in compatible shape.

# Build CNN

- Define layers.
- Set optimizer and loss function. (Use Adam optimizer and categorical crossentropy.)


# Modelling and Evaluation

- Fit and evaluate model and print confusion matrix

# Visualize Predictions

- Visualize predictions for x_test[2], x_test[3], x_test[33], x_test[36], x_test[59]